# Generate KD Cache: WikiText-103, L=1024 (Raw Text)

**Purpose:** Anti-repetition + long-context coherence training

This notebook generates a KD cache using **Qwen3-32B** as teacher on **WikiText-103** raw text.

**Key settings:**
- `template_mode=none` (raw text, no chat template)
- `L=1024` (long sequences for coherence)
- Clean Wikipedia text (reduces repetition patterns)

**Requirements:**
- GPU with 80GB+ VRAM (A100 80GB, H100)
- Or 40GB+ with 4-bit quantization (batch_size=1)

**Estimated storage:** ~15-25GB for 8K sequences at L=1024

In [ ]:
# ============================================================
# CHECK GPU
# ============================================================

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

In [ ]:
# ============================================================
# SETUP: Clone repo and install dependencies
# ============================================================

!git clone https://github.com/anemll/qwen3_apple_style_2bit_qat_lora.git 2>/dev/null || echo "Repo already exists"
%cd qwen3_apple_style_2bit_qat_lora
!git pull origin main

!pip install -q transformers accelerate datasets sentencepiece

In [ ]:
# ============================================================
# CONFIG - WikiText-103, L=1024, Raw Text
# ============================================================

# Teacher model - Qwen3-32B (or 0.6B for cheaper runs)
MODEL_NAME = 'Qwen/Qwen3-32B'
# MODEL_NAME = 'Qwen/Qwen3-0.6B'  # Cheaper alternative

# Dataset - WikiText-103 for clean long-form text
DATASET_NAME = 'Salesforce/wikitext'
DATASET_SUBSET = 'wikitext-103-v1'
DATASET_SPLIT = 'train'
DATASET_FORMAT = 'plain'  # Raw text, no chat template

# Cache parameters - L=1024 for long-context coherence
TOP_K = 128           # Number of top-k logits to cache
RAND_NEG = 64         # Random negatives
MAX_LENGTH = 1024     # Long sequences for coherence training
NUM_SEQUENCES = 8000  # Start with 8K (~8.2M tokens)
SHARD_SIZE = 100      # Smaller shards for L=1024 (larger tensors)

# Batch size for 80GB A100 with 32B @ L=1024
# Tested: batch=16 uses ~75/80GB VRAM
BATCH_SIZE = 16

# No thinking variants - raw text only
ENABLE_THINKING = 'none'

# Output cache name
CACHE_NAME = f"wikitext103_32B_L{MAX_LENGTH}_K{TOP_K}_R{RAND_NEG}_N{NUM_SEQUENCES}"
CACHE_DIR = f"caches/{CACHE_NAME}"

print(f"="*60)
print(f"KD Cache Configuration: WikiText-103 L=1024")
print(f"="*60)
print(f"Teacher model:    {MODEL_NAME}")
print(f"Dataset:          {DATASET_NAME} / {DATASET_SUBSET}")
print(f"Format:           {DATASET_FORMAT} (raw text, no template)")
print(f"Top-K:            {TOP_K}")
print(f"Random negatives: {RAND_NEG}")
print(f"Max length:       {MAX_LENGTH}")
print(f"Sequences:        {NUM_SEQUENCES} (~{NUM_SEQUENCES * MAX_LENGTH / 1e6:.1f}M tokens)")
print(f"Batch size:       {BATCH_SIZE} (~75GB VRAM on A100-80GB)")
print(f"Shard size:       {SHARD_SIZE}")
print(f"Output:           {CACHE_DIR}")
print(f"="*60)
print(f"\nEstimated storage: ~{NUM_SEQUENCES * MAX_LENGTH * TOP_K * 4 / 1e9:.1f}GB")

In [ ]:
# ============================================================
# OPTIONAL: Mount Google Drive for saving cache
# ============================================================

SAVE_TO_DRIVE = True  # Set to False if not using Colab/Drive

if SAVE_TO_DRIVE:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_CACHE_DIR = '/content/drive/MyDrive/qwen3_caches'
        !mkdir -p {DRIVE_CACHE_DIR}
        print(f"Google Drive mounted. Will save to: {DRIVE_CACHE_DIR}")
    except:
        print("Not running in Colab or Drive mount failed. Saving locally only.")
        SAVE_TO_DRIVE = False

In [ ]:
# ============================================================
# GENERATE KD CACHE
# ============================================================
# WikiText-103 with L=1024 for long-context coherence
#
# Tested on Colab A100 80GB:
#   - 32B, batch=16: ~60 min for 8K sequences (~42s/shard)
# ============================================================

%cd /content/qwen3_apple_style_2bit_qat_lora

!python scripts/precompute_teacher_topk.py \
    --teacher_model_name_or_path {MODEL_NAME} \
    --dataset_name {DATASET_NAME} \
    --dataset_config_name {DATASET_SUBSET} \
    --dataset_split {DATASET_SPLIT} \
    --dataset_format {DATASET_FORMAT} \
    --enable_thinking {ENABLE_THINKING} \
    --max_length {MAX_LENGTH} \
    --topk {TOP_K} \
    --rand_neg {RAND_NEG} \
    --num_sequences {NUM_SEQUENCES} \
    --output_dir {CACHE_DIR} \
    --batch_size {BATCH_SIZE} \
    --shard_size {SHARD_SIZE} \
    --dtype bf16 \
    --device auto

In [ ]:
# ============================================================
# VERIFY CACHE
# ============================================================

import os
import json

if os.path.isdir(CACHE_DIR):
    # Count shards
    shards = [f for f in os.listdir(CACHE_DIR) if f.startswith('shard_')]
    print(f"[cache] Generated {len(shards)} shards")
    
    # Check meta.json
    meta_path = os.path.join(CACHE_DIR, 'meta.json')
    if os.path.exists(meta_path):
        with open(meta_path) as f:
            meta = json.load(f)
        print(f"[cache] Meta info:")
        for k, v in meta.items():
            print(f"  - {k}: {v}")
    
    # Calculate size
    total_size = sum(os.path.getsize(os.path.join(CACHE_DIR, f)) for f in os.listdir(CACHE_DIR))
    print(f"[cache] Total size: {total_size / (1024**3):.2f} GB")
else:
    print(f"[cache] ERROR: {CACHE_DIR} not found")

In [ ]:
# ============================================================
# INSPECT A SAMPLE SHARD
# ============================================================

import torch
import os

shard_files = sorted([f for f in os.listdir(CACHE_DIR) if f.startswith('shard_')])
if shard_files:
    shard_path = os.path.join(CACHE_DIR, shard_files[0])
    shard = torch.load(shard_path, map_location='cpu')
    
    print(f"Shard: {shard_files[0]}")
    print(f"Keys: {list(shard.keys())}")
    
    # Expected shapes for L=1024, K=128:
    # input_ids: [SHARD_SIZE, 1024]
    # topk_indices: [SHARD_SIZE, 1024, 128]
    # topk_probs: [SHARD_SIZE, 1024, 128]
    
    if 'input_ids' in shard:
        print(f"\ninput_ids shape: {shard['input_ids'].shape}  (expected: [{SHARD_SIZE}, {MAX_LENGTH}])")
    if 'topk_indices' in shard:
        print(f"topk_indices shape: {shard['topk_indices'].shape}  (expected: [{SHARD_SIZE}, {MAX_LENGTH}, {TOP_K}])")
    if 'topk_probs' in shard:
        print(f"topk_probs shape: {shard['topk_probs'].shape}  (expected: [{SHARD_SIZE}, {MAX_LENGTH}, {TOP_K}])")

In [ ]:
# ============================================================
# SAVE TO GOOGLE DRIVE
# ============================================================

if SAVE_TO_DRIVE and os.path.isdir(CACHE_DIR):
    print(f"[save] Copying {CACHE_NAME} to Google Drive...")
    !rsync -ah --info=progress2 {CACHE_DIR}/ {DRIVE_CACHE_DIR}/{CACHE_NAME}/
    
    # Verify
    gd_path = f"{DRIVE_CACHE_DIR}/{CACHE_NAME}"
    if os.path.isdir(gd_path):
        num_files = len(os.listdir(gd_path))
        print(f"[save] Successfully saved to Google Drive: {num_files} files")
    else:
        print(f"[save] ERROR: Failed to copy to Google Drive")
else:
    print("[save] Skipping Google Drive save")

## Batch Size Reference (Colab A100 80GB)

32B @ L=1024 in bf16:
- **batch_size=16**: ~75/80GB VRAM (tested on Colab A100 High-Memory)

### For different GPUs:

| GPU | VRAM | Recommended batch_size |
|-----|------|------------------------|
| Colab A100 80GB | 80GB | 16 |
| A100 40GB | 40GB | 4-8 |
| Colab T4 | 16GB | Use 0.6B teacher |

### If you get OOM errors:

```python
BATCH_SIZE = 8   # Try halving
# or
MODEL_NAME = 'Qwen/Qwen3-0.6B'  # Smaller teacher
BATCH_SIZE = 32
```

## Summary

**Cache purpose:** Anti-repetition + long-context coherence

**Cache generated:** `wikitext103_32B_L1024_K128_R64_N8000`

**Contains:**
- 8K sequences x 1024 tokens = ~8.2M tokens of teacher supervision
- Top-128 teacher logits from Qwen3-32B
- Raw text format (no chat template, no thinking tags)
- Clean Wikipedia text for coherence patterns

**To use in LoRA recovery training:**
```bash
python scripts/train_recovery_lora.py \
    --kd-cache-dir caches/wikitext103_32B_L1024_K128_R64_N8000 \
    --seq-len 1024 \
    --lora-mode kd \
    ...
```

**Load from Google Drive:**
```bash
rsync -ah --info=progress2 /content/drive/MyDrive/qwen3_caches/wikitext103_32B_L1024_K128_R64_N8000/ caches/wikitext103_32B_L1024_K128_R64_N8000/
```